Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [2]:
library('tidymodels')
library('ISLR2')

In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [3]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [4]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [5]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [6]:
mod = logistic_reg() |>
    set_engine('glm') |>
    set_mode('classification')

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [7]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9671111


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [8]:
folds = vfold_cv(Default_train, v = 10, strata = default)
folds |> glimpse()

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [9]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.9734444,10,0.001266320,Preprocessor1_Model1
brier_class,binary,0.0216338,10,0.001021663,Preprocessor1_Model1
roc_auc,binary,0.9455921,10,0.007592827,Preprocessor1_Model1


**❓How does the model accuracy compare to the naive model from above?**
> Look at the mean column for accuracy (0.973). It is greater than 0.967 (estimate from the naive model), so it outperforms the naive model.

Complete the cell below to evaluate a model that also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [10]:
rec = recipe(default ~ income + balance + student, data = Default_train) |>
    step_dummy(student)
    #prep() --> I get an error when I try to use prep()

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97288889,10,0.001327146,Preprocessor1_Model1
brier_class,binary,0.02158467,10,0.001044476,Preprocessor1_Model1
roc_auc,binary,0.94555024,10,0.007714667,Preprocessor1_Model1


**❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?**

> No, the model that includes the student variable **does not improve** upon the performance over the first model. So student does not appear to add predictive value beyond what income and balance already provide.

Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

**❓Does our model outperform a naive model?**

> Yes! The naive model had an accuracy of 0.967. The logistic regression model has an accuracy of approximately 0.976 on the test set, meaning it **does outperform the naive model**.

In [12]:
mod_test = mod |> fit(default ~ income + balance, data = Default_train)

test_predictions = predict(mod_test, new_data = Default_test, type = "class") |>
  bind_cols(Default_test)

test_predictions |> head()

.pred_class,default,student,balance,income
<fct>,<fct>,<fct>,<dbl>,<dbl>
No,No,No,0.0000,29275.27
No,No,No,1112.9684,23810.17
No,No,No,653.1184,39489.59
No,No,No,837.2626,51471.77
No,No,Yes,1275.5506,15887.47
No,No,No,766.2344,46478.29


In [13]:
test_predictions |> 
  accuracy(truth = default, estimate = .pred_class)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.976
